<h1>Generate Mistrust Scores for All Patients</h1>
- Supervised Machine Learning (binary chartevents features) to predict whether "noncompliance" appears in the notes
- Supervised Machine Learning (binary chartevents features) to predict whether the patient or family consents/declines autopsy
- Off the shelf sentiment analysis of the notes

In [ ]:
from collections import Counter
from collections import defaultdict
import pickle
import numpy as np
import pandas as pd
import psycopg2
import random
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import tqdm
from time import gmtime, strftime
# Import libraries
from datetime import timedelta
import os
from pandas_gbq import read_gbq
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import time
from collections import defaultdict
import pylab as pl
from scipy.stats import mannwhitneyu

# Make pandas dataframes prettier
from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
from google.cloud.bigquery import Client

In [ ]:
# authenticate
auth.authenticate_user()

In [ ]:
# Set up environment variables
project_id = 'CHANGE-ME'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

bq_client = bigquery.Client(project=project_id)

# Modified run_query function using BigQuery client
def run_query(query: str):
    query_job = bq_client.query(query)
    return query_job.to_dataframe(create_bqstorage_client=True)

'''
# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return read_gbq(
      query,
      project_id=project_id,
      dialect='standard')
'''

# set the dataset
# if you want to use the demo, change this to mimic_demo
hosp_dataset_4 = 'mimiciv_3_1_hosp'
icu_dataset_4 = 'mimiciv_3_1_icu'
derived_dataset_4 = 'mimiciv_3_1_derived'
derived_dataset_3 = 'mimiciii_derived'
clinical_dataset_3 = 'mimiciii_clinical'
note_dataset_3 = 'mimiciii_notes'

#indicate whether to use the mimic iii versions of the tables with their own columns

#indicate whether to run a limited sample size for testing purposes
limited_sample = False


# test it works
df = run_query("""
SELECT subject_id
FROM `physionet-data.mimiciii_clinical.chartevents`
WHERE subject_id = 40080
""")
df.head()

,subject_id
0,40080
1,40080
2,40080
3,40080
4,40080


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h1>Load Data from MIMIC</h1>

In [ ]:
# Read interpersonal interaction variables from chartevents

relevant_labels = '''
Family Communication
Follows Commands
Education Barrier
Education Learner
Education Method
Education Readiness
Education Topic #1
Education Topic #2
Pain
Pain Level
Pain Level (Rest)
Pain Assess Method
Restraint
Restraint Type
Restraint (Non-violent)
Restraint Ordered (Non-violent)
Restraint Location
Reason For Restraint
Spiritual Support
Support Systems
State
Behavior
Behavioral State
Reason For Restraint
Stress
Safety
Safety Measures_U_1
Family
Patient/Family Informed
Pt./Family Informed
Health Care Proxy
BATH
bath
Bath
Bed Bath
bed bath
bed bath
Bedbath
CHG Bath
Skin Care
Judgement
Family Meeting held
Emotional / physical / sexual harm by partner or close relation
Verbal Response
Side Rails
Orientation
RSBI Deferred
Richmond-RAS Scale
Riker-SAS Scale
Status and Comfort
Teaching directed toward
Consults
Social work consult
Sitter
security
safety
headache
hairwashed
observer
'''

# Build SQL filter conditions using case-insensitive partial match
matches_text = []
for rl in relevant_labels.strip().split('\n'):
    rl = rl.strip()
    if rl:
        matches_text.append(f"LOWER(label) LIKE '%{rl.lower()}%'")

# Join conditions with OR
matches = ' OR '.join(matches_text)
#matches = ' or '.join(matches_text)

# Log query start time
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

# Compose query: join `chartevents` with `d_items` to access human-readable labels
chartevents_query = f'''
SELECT DISTINCT hadm_id, label, value
FROM physionet-data.mimiciii_clinical.chartevents c
JOIN physionet-data.mimiciii_clinical.d_items i ON i.itemid = c.itemid
WHERE {matches}
'''

#chartevents_query = 'select distinct hadm_id,label,value from physionet-data.mimiciii_clinical.chartevents c JOIN physionet-data.mimiciii_clinical.d_items i on i.itemid=c.itemid where (%s)' % matches
# Execute query
chartevents = run_query(chartevents_query)
chartevents.to_csv("chartevents.csv", index=False)

# Log query end time
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

print(type(chartevents))
#chartevents.head()

2025-05-04 21:15:53
2025-05-04 21:16:07
<class 'pandas.core.frame.DataFrame'>


In [ ]:
dtype_mapping = {
    "hadm_id": "Int64",  # Nullable integer
    "label": "string",
    "value": "string"
}

chartevents = pd.read_csv("chartevents.csv", dtype=dtype_mapping)

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
chartevents_cache = chartevents

In [ ]:
# Log the start time of the query
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

# Construct SQL query to extract clinical notes (excluding erroneous entries)
notes_query = f"""
SELECT DISTINCT
    n.hadm_id,
    n.category,
    n.text,
    n.chartdate,
    n.charttime
FROM physionet-data.{note_dataset_3}.noteevents n
WHERE iserror IS NULL      -- Only include notes without errors
  AND hadm_id IS NOT NULL  -- Ensure admission ID is present
;
"""

# Run the query and store results in a DataFrame
notes = run_query(notes_query)

# Log the end time of the query
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

#notes.head()

2025-05-04 21:20:33
2025-05-04 21:21:17


In [ ]:
notes.to_csv("notes.csv", index=False)

In [ ]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define save paths
drive_path = "/content/drive/MyDrive/mimic_data"
os.makedirs(drive_path, exist_ok=True)

# Save DataFrames to CSV in Google Drive
chartevents.to_csv(os.path.join(drive_path, "chartevents.csv"), index=False)
notes.to_csv(os.path.join(drive_path, "notes.csv"), index=False)


Mounted at /content/drive


In [ ]:
from google.colab import drive
import pandas as pd
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define path
drive_path = "/content/drive/MyDrive/mimic_data"

# Define dtype mappings
chartevents_dtypes = {
    "hadm_id": "Int64",
    "label": "string",
    "value": "string"
}

notes_dtypes = {
    "hadm_id": "Int64",
    "category": "string",
    "text": "string",
    "chartdate": "string",
    "charttime": "string"
}

# Load CSVs with dtype enforcement
chartevents = pd.read_csv(os.path.join(drive_path, "chartevents.csv"), dtype=chartevents_dtypes)
notes = pd.read_csv(os.path.join(drive_path, "notes.csv"), dtype=notes_dtypes)


<h1> Extract Features and Labels</h1>

In [ ]:
# Extract features from chartevents

chartevents_features = {}
for hadm_id,rows in tqdm.tqdm(chartevents.groupby('hadm_id')):
    feats = {}
    for i,row in rows.iterrows():
        label = row.label.lower()

        if row.value is None:
            val = 'none'
        else:
            val = row.value.lower()


        if 'reason for restraint' in label:

            if (val == 'not applicable') or (val == 'none'):
                val = 'none'
            elif ('threat' in val) or ('acute risk of' in val):
                val = 'threat of harm'
            elif ('confusion' in val) or ('delirium' in val) or (val == 'impaired judgment') or (val == 'sundowning'):
                val = 'confusion/delirium'
            elif ('occurence' in val) or (val == 'severe physical agitation') or (val == 'violent/self des'):
                val = 'prescence of violence'
            elif (val == 'ext/txinterfere') or (val == 'protection of lines and tubes') or (val == 'treatment interference'):
                val = 'treatment interference'
            elif 'risk for fall' in val:
                val = 'risk for falls'
            else:
                val = val

            feats[('reason for restraint', val)] = 1

        elif 'restraint location' in label:

            if val == 'none':
                val = 'none'
            elif '4 point rest' in val:
                val = '4 point restraint'
            else:
                val = 'some restraint'

            feats[('restraint location', val)] = 1

        elif 'restraint device' in label:

            if 'sitter' in val:
                val = 'sitter'
            elif 'limb' in val:
                val = 'limb'
            else:
                val = val

            feats[('restraint device', val)] = 1

        elif 'bath' in label:
            if 'part' in label:
                val = 'partial'
            elif 'self' in val:
                val = 'self'
            elif 'refused' in val:
                val = 'refused'
            elif 'shave' in val:
                val = 'shave'
            elif 'hair' in val:
                val = 'hair'
            elif 'none' in val:
                val = 'none'
            else:
                val = 'done'

            feats[('bath', val)] = 1

        elif label in ['behavior', 'behavioral state']:
            #feats[('behavior', val)] = 1
            pass

        elif label.startswith('pain level'):
            feats[('pain level', val)] = 1

        elif label.startswith('pain management'):
            #feats[('pain management', val)] = 1
            pass
        elif label.startswith('pain type'):
            #feats[('pain type', val)] = 1
            pass
        elif label.startswith('pain cause'):
            #feats[('pain cause', val)] = 1
            pass
        elif label.startswith('pain location'):
            #feats[('pain location', val)] = 1
            pass

        elif label.startswith('education topic'):
            feats[('education topic', val)] = 1

        elif label.startswith('safety measures'):
            feats[('safety measures', val)] = 1

        elif label.startswith('side rails'):
            feats[('side rails', val)] = 1

        elif label.startswith('status and comfort'):
            feats[('status and comfort', val)] = 1

        elif 'informed' in label:
            feats[('informed', val)] = 1
        else:

            if type(row.value) == type(''):
                # extract phrase
                featname = (row.label.lower(), row.value.lower())
                value = 1.0
                feats[featname] = value
            elif row.value is None:
                featname = (row.label.lower(),'none')
                value = 1.0
                feats[featname] = value
            else:
                featname = (row.label.lower(),)
                value = value
                feats[featname] = value
                pass


    chartevents_features[hadm_id] = feats


100%|██████████| 54510/54510 [04:47<00:00, 189.32it/s]


In [ ]:
with open(os.path.join(drive_path, "chartevents_features.pkl"), "wb") as f:
    pickle.dump(chartevents_features, f)

In [ ]:
# LABEL: noncompliance in notes

mistrust_ids = []
for hadm_id,rows in tqdm.tqdm(notes.groupby('hadm_id')):
    # This is customizable to various note-based definitions of what to look for
    mistrust = False
    for text in rows.text.values:
        if 'noncompliant' in text.lower():
            mistrust = True

    # add the ID
    if mistrust:
        mistrust_ids.append(hadm_id)


# binary labels
trust_labels_noncompliance = {hadm_id:'trust' for hadm_id in chartevents['hadm_id'].values}
for hadm_id in mistrust_ids:
    trust_labels_noncompliance[int(hadm_id)] = 'mistrust'

print('patients labeled as    trustful:', len([y for y in trust_labels_noncompliance.values() if y=='trust'   ]))
print('patients labeled as mistrustful:', len([y for y in trust_labels_noncompliance.values() if y=='mistrust']))

100%|██████████| 58361/58361 [00:20<00:00, 2791.88it/s]


patients labeled as    trustful: 54030
patients labeled as mistrustful: 484


In [ ]:
# Identify hospital admissions with mentions of "noncompliant" in notes
mistrust_ids = []
for hadm_id, rows in tqdm.tqdm(notes.groupby('hadm_id')):
    mistrust = any('noncompliant' in text.lower() for text in rows.text.values if isinstance(text, str))
    if mistrust:
        mistrust_ids.append(hadm_id)

# Create binary labels: default is "trust", override with "mistrust" if matched
trust_labels_noncompliance = {int(hadm_id): 'trust' for hadm_id in chartevents['hadm_id'].dropna().unique()}
for hadm_id in mistrust_ids:
    trust_labels_noncompliance[int(hadm_id)] = 'mistrust'

# Print summary
print('patients labeled as    trustful:', sum(1 for y in trust_labels_noncompliance.values() if y == 'trust'))
print('patients labeled as mistrustful:', sum(1 for y in trust_labels_noncompliance.values() if y == 'mistrust'))

# Save to disk for hotloading later
with open(os.path.join(drive_path, "trust_labels_noncompliance.pkl"), "wb") as f:
    pickle.dump(trust_labels_noncompliance, f)

100%|██████████| 58361/58361 [00:15<00:00, 3823.79it/s]


patients labeled as    trustful: 54030
patients labeled as mistrustful: 484


In [ ]:
# LABEL: whether patient got autopsy

autopsy_consent = []
autopsy_decline = []
for hadm_id,rows in tqdm.tqdm(notes.groupby('hadm_id')):
    consented = False
    declined = False
    for text in rows.text.values:
        for line in text.lower().split('\n'):
            if 'autopsy' in line:
                if 'decline' in line:
                    declined = True
                if 'not consent' in line:
                    declined = True
                if 'refuse' in line:
                    declined = True
                if 'denied' in line:
                    declined = True

                if 'consent' in line:
                    consented = True
                if 'agree' in line:
                    consented = True
                if 'request' in line:
                    consented = True

    # probably some "declined donation but consented to autopsy" or something confusing. just ignore hard cases
    if consented and declined:
        continue

    if consented:
        autopsy_consent.append(hadm_id)
    if declined:
        autopsy_decline.append(hadm_id)

# binary labels
trust_labels_autopsy = {}
for hadm_id in autopsy_consent:
    trust_labels_autopsy[int(hadm_id)] = 'mistrust'
for hadm_id in autopsy_decline:
    trust_labels_autopsy[int(hadm_id)] = 'trust'

print('patients labeled as    trustful:', len([y for y in trust_labels_autopsy.values() if y=='trust'   ]))
print('patients labeled as mistrustful:', len([y for y in trust_labels_autopsy.values() if y=='mistrust']))

100%|██████████| 58361/58361 [00:29<00:00, 1993.04it/s]


patients labeled as    trustful: 739
patients labeled as mistrustful: 270


In [ ]:
# Lists to hold admission IDs based on autopsy consent/decline status
autopsy_consent = []
autopsy_decline = []

# Iterate over notes grouped by admission ID
for hadm_id, rows in tqdm.tqdm(notes.groupby('hadm_id')):
    consented = False
    declined = False

    # Search for relevant phrases in note text
    for text in rows.text.values:
        if not isinstance(text, str):
            continue
        for line in text.lower().split('\n'):
            if 'autopsy' in line:
                if any(kw in line for kw in ['decline', 'not consent', 'refuse', 'denied']):
                    declined = True
                if any(kw in line for kw in ['consent', 'agree', 'request']):
                    consented = True

    # Skip conflicting or ambiguous notes
    if consented and declined:
        continue
    if consented:
        autopsy_consent.append(hadm_id)
    if declined:
        autopsy_decline.append(hadm_id)

# Assign trust labels: consent implies mistrust, decline implies trust
trust_labels_autopsy = {}
for hadm_id in autopsy_consent:
    trust_labels_autopsy[int(hadm_id)] = 'mistrust'
for hadm_id in autopsy_decline:
    trust_labels_autopsy[int(hadm_id)] = 'trust'

# Print summary
print('patients labeled as    trustful:', len([y for y in trust_labels_autopsy.values() if y == 'trust']))
print('patients labeled as mistrustful:', len([y for y in trust_labels_autopsy.values() if y == 'mistrust']))

# Save labels to disk for reuse
with open(os.path.join(drive_path, "trust_labels_autopsy.pkl"), "wb") as f:
    pickle.dump(trust_labels_autopsy, f)

100%|██████████| 58361/58361 [00:36<00:00, 1587.19it/s]

patients labeled as    trustful: 739
patients labeled as mistrustful: 270


<h1>Supervised Learning to Classify Trust-based Outcomes</h1>

In [ ]:
# Split a list of IDs into training and test sets based on a given ratio
def data_split(ids, ratio=0.7):
    random.shuffle(ids)
    split_idx = int(len(ids) * ratio)
    train = ids[:split_idx]
    test = ids[split_idx:]
    return train, test


# Wrapper function to compute binary classification stats with score difference
def compute_stats(task, pred, P, ref, labels_map):
    # Use score difference between positive and negative class
    scores = P[:, 1] - P[:, 0]
    return compute_stats_binary(task, pred, scores, ref, labels_map)


# Compute evaluation metrics for binary classification
def compute_stats_binary(task, pred, scores, ref, labels):
    # Ensure prediction matches score thresholding
    assert all((scores > 0).astype(int) == pred), "Mismatch between scores and predictions"

    n_classes = len(set(labels.values()))
    assert n_classes == 2, 'Only binary classification supported for this function'

    conf = np.zeros((2, 2), dtype='int32')
    for p, r in zip(pred, ref):
        conf[p][r] += 1

    print("Confusion matrix:\n", conf)

    tp = conf[1, 1]
    tn = conf[0, 0]
    fp = conf[1, 0]
    fn = conf[0, 1]

    precision   = tp / (tp + fp + 1e-9)
    recall      = tp / (tp + fn + 1e-9)
    sensitivity = recall  # same as recall
    specificity = tn / (tn + fp + 1e-9)
    f1          = (2 * precision * recall) / (precision + recall + 1e-9)
    accuracy    = (tp + tn) / (tp + tn + fp + fn + 1e-9)

    print(f"\tspecificity: {specificity:.3f}")
    print(f"\tsensitivity: {sensitivity:.3f}")

    auc = None
    if len(set(ref)) == 2:
        auc = sklearn.metrics.roc_auc_score(ref, scores)
        print(f"\tauc:        {auc:.3f}")

    print(f"\taccuracy:   {accuracy:.3f}")
    print(f"\tprecision:  {precision:.3f}")
    print(f"\trecall:     {recall:.3f}")
    print(f"\tf1:         {f1:.3f}")

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'sensitivity': sensitivity,
        'specificity': specificity
    }


# Run a classification model and evaluate it with prediction metrics
def classification_results(svm, labels_map, X, Y, task):
    # Get raw decision function scores from the classifier
    decision_scores = svm.decision_function(X)

    # Normalize decision function output to 2D probability-like scores for binary classification
    if len(labels_map) == 2:
        m = X.shape[0]
        P = np.zeros((m, 2))
        P[:, 0] = -decision_scores
        P[:, 1] = decision_scores
    else:
        P = decision_scores

    # Predict label by selecting the highest score
    predicted_labels = P.argmax(axis=1)

    print(task)
    return compute_stats(task, predicted_labels, P, Y, labels_map)


In [ ]:
from sklearn.feature_extraction import DictVectorizer

# Fit vectorizer for chartevents features
vect = DictVectorizer()
vect.fit(chartevents_features.values())

print('num_features:', len(vect.get_feature_names_out()))

num_features: 620


In [ ]:
# display informative features

def classification_analyze(task, vect, clf, labels_map, num_feats=10):
    ind2feat =  { i:f for f,i in vect.vocabulary_.items() }

    labels = [label for label,i in sorted(labels_map.items(), key=lambda t:t[1])]
    coef_ = clf.coef_

    informative_feats = np.argsort(coef_)

    neg_features = informative_feats[0,:num_feats ]
    pos_features = informative_feats[0,-num_feats:]

    # display what each feature is
    print('POS %s' % label)
    for feat in reversed(pos_features):
        val = coef_[0,feat]
        word = ind2feat[feat]
        if val > 1e-4:
            print('\t%-25s: %7.4f' % (word,val))
        else:
            break
    print('NEG %s' % label)
    for feat in reversed(neg_features):
        val = coef_[0,feat]
        word = ind2feat[feat]
        if -val > 1e-4:
            print('\t%-25s: %7.4f' % (word,val))
        else:
            continue
    print('\n')

In [ ]:
# vectorize task-specific labels
trust_Y_vect = {'mistrust': 1, 'trust': 0}
print(trust_Y_vect)

{'mistrust': 1, 'trust': 0}


In [ ]:
# CLASSIFIER: noncompliance

noncompliance_cohort = list(set(trust_labels_noncompliance.keys()) & set(chartevents['hadm_id'].values))
print('patients:', len(noncompliance_cohort))

# train/test split
noncompliance_train_ids, noncompliance_test_ids = data_split(noncompliance_cohort)

# select pre-computed features
noncompliance_train_features = [chartevents_features[hadm_id] for hadm_id in noncompliance_train_ids]
noncompliance_test_features  = [chartevents_features[hadm_id] for hadm_id in noncompliance_test_ids ]

# vectorize features
noncompliance_train_X = vect.transform(noncompliance_train_features)
noncompliance_test_X  = vect.transform(noncompliance_test_features)

# select labels
noncompliance_train_Y = [trust_Y_vect[trust_labels_noncompliance[hadm_id]] for hadm_id in noncompliance_train_ids]
noncompliance_test_Y  = [trust_Y_vect[trust_labels_noncompliance[hadm_id]] for hadm_id in noncompliance_test_ids ]

# fit model
#noncompliance_svm = LogisticRegression(C=0.1, penalty='l1', tol=0.01)
noncompliance_svm = LogisticRegression(
    C=0.1,
    penalty='l1',
    tol=0.01,
    solver='liblinear'  # Must use 'liblinear' or 'saga' for L1 penalty
)
noncompliance_svm.fit(noncompliance_train_X, noncompliance_train_Y)
print(noncompliance_svm)

# evaluate model
classification_results(noncompliance_svm, trust_Y_vect, noncompliance_train_X, noncompliance_train_Y, 'train: noncompliance')
classification_results(noncompliance_svm, trust_Y_vect,  noncompliance_test_X,  noncompliance_test_Y, 'test:  noncompliance')

# most informative features
classification_analyze('noncompliance', vect, noncompliance_svm, trust_Y_vect, num_feats=15)


patients: 54510
LogisticRegression(C=0.1, penalty='l1', solver='liblinear', tol=0.01)
train: noncompliance
Confusion matrix:
 [[37815   342]
 [    0     0]]
	specificity: 1.000
	sensitivity: 0.000
	auc:        0.714
	accuracy:   0.991
	precision:  0.000
	recall:     0.000
	f1:         0.000
test:  noncompliance
Confusion matrix:
 [[16215   138]
 [    0     0]]
	specificity: 1.000
	sensitivity: 0.000
	auc:        0.688
	accuracy:   0.992
	precision:  0.000
	recall:     0.000
	f1:         0.000
POS safety measures_u_1
	('riker-sas scale', 'agitated'):  0.6826
	('riker-sas scale', 'very agitated'):  0.2988
	('education readiness', 'no'):  0.2794
	('side rails', 'all rails up (restraint)'):  0.2407
	('pain level', '0-none') :  0.1738
	('pain level', '7-mod to severe'):  0.1129
	('education topic', 'medications'):  0.0915
	('pain level', 'none')   :  0.0772
	('headache', 'not present'):  0.0741
	('consults', 'social work'):  0.0638
	('orientation', 'oriented x 2'):  0.0601
	('pain level', '

In [ ]:
import joblib

# Mount Google Drive
drive_path = "/content/drive/MyDrive/mimic_data"
os.makedirs(drive_path, exist_ok=True)

# Save the trained model to Google Drive
joblib.dump(noncompliance_svm, os.path.join(drive_path, "noncompliance_svm.pkl"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['/content/drive/MyDrive/mimic_data/noncompliance_svm.pkl']

In [ ]:
noncompliance_svm = joblib.load(os.path.join(drive_path, "noncompliance_svm.pkl"))

In [ ]:
# CLASSIFIER: autopsy

autopsy_cohort = list(set(trust_labels_autopsy.keys()) & set(chartevents['hadm_id'].values))
print('patients:', len(autopsy_cohort))

# train/test split
autopsy_train_ids, autopsy_test_ids = data_split(autopsy_cohort)

# select pre-computed features
autopsy_train_features = [chartevents_features[hadm_id] for hadm_id in autopsy_train_ids]
autopsy_test_features  = [chartevents_features[hadm_id] for hadm_id in autopsy_test_ids ]

# vectorize features
autopsy_train_X = vect.transform(autopsy_train_features)
autopsy_test_X  = vect.transform(autopsy_test_features)

# select labels
autopsy_train_Y = [trust_Y_vect[trust_labels_autopsy[hadm_id]] for hadm_id in autopsy_train_ids]
autopsy_test_Y  = [trust_Y_vect[trust_labels_autopsy[hadm_id]] for hadm_id in autopsy_test_ids ]

# fit model
autopsy_svm = LogisticRegression(
    C=0.1,
    penalty='l1',
    tol=0.01,
    solver='liblinear'  # Must use 'liblinear' or 'saga' for L1 penalty
)
autopsy_svm.fit(autopsy_train_X, autopsy_train_Y)
print(autopsy_svm)

# evaluate model
classification_results(autopsy_svm, trust_Y_vect, autopsy_train_X, autopsy_train_Y, 'train: autopsy')
classification_results(autopsy_svm, trust_Y_vect,  autopsy_test_X,  autopsy_test_Y, 'test:  autopsy')

# most informative features
classification_analyze('trust', vect, autopsy_svm, trust_Y_vect, num_feats=15)

patients: 997
LogisticRegression(C=0.1, penalty='l1', solver='liblinear', tol=0.01)
train: autopsy
Confusion matrix:
 [[516 181]
 [  0   0]]
	specificity: 1.000
	sensitivity: 0.000
	auc:        0.616
	accuracy:   0.740
	precision:  0.000
	recall:     0.000
	f1:         0.000
test:  autopsy
Confusion matrix:
 [[215  85]
 [  0   0]]
	specificity: 1.000
	sensitivity: 0.000
	auc:        0.588
	accuracy:   0.717
	precision:  0.000
	recall:     0.000
	f1:         0.000
POS safety measures_u_1
	('riker-sas scale', 'very sedated'):  0.1783
	('education barrier', 'medicated'):  0.1246
	('restraints evaluated', 'restraintreapply'):  0.0008
NEG safety measures_u_1
	('family communication', 'family talked to md'): -0.1171
	('side rails', '3 rails up'): -0.1704
	('support systems', 'children'): -0.1779
	('pain present', 'no')   : -0.2406




In [ ]:
import joblib

# Mount Google Drive
drive_path = "/content/drive/MyDrive/mimic_data"
os.makedirs(drive_path, exist_ok=True)

# Save the trained model to Google Drive
joblib.dump(autopsy_svm, os.path.join(drive_path, "autopsy_svm.pkl"))

['/content/drive/MyDrive/mimic_data/autopsy_svm.pkl']

In [ ]:
autopsy_svm = joblib.load(os.path.join(drive_path, "autopsy_svm.pkl"))

In [ ]:
# Predict scores for all patients and save to file

# ordering of all chartevents features
chartevents_ids = set(chartevents['hadm_id'].values)
chartevents_X = vect.transform([chartevents_features[hadm_id] for hadm_id in chartevents_ids])

# Save ranking (i.e. confidence from trust classifier) of all patients on the noncompliance metric
with open('../data/mistrust_noncompliant.pkl', 'wb') as f:
    noncompliance_scores = dict(zip(chartevents_ids,noncompliance_svm.decision_function(chartevents_X)))
    pickle.dump(noncompliance_scores, f)
print('DONE: noncompliance scores')

# Save ranking (i.e. confidence from trust classifier) of all patients on the autopsy metric
with open('../data/mistrust_autopsy.pkl', 'wb') as f:
    autopsy_scores = dict(zip(chartevents_ids,autopsy_svm.decision_function(chartevents_X)))
    pickle.dump(autopsy_scores, f)
print('DONE: autopsy scores')

DONE: noncompliance scores
DONE: autopsy scores


In [ ]:
# Mount Google Drive
base_path = "/content/drive/MyDrive/mimic_data/data"
os.makedirs(base_path, exist_ok=True)

# Prepare feature matrix for all patients
chartevents_ids = set(chartevents['hadm_id'].values)
chartevents_X = vect.transform([chartevents_features[hadm_id] for hadm_id in chartevents_ids])

# Save noncompliance scores
noncompliance_scores = dict(zip(chartevents_ids, noncompliance_svm.decision_function(chartevents_X)))
with open(os.path.join(base_path, "mistrust_noncompliant.pkl"), "wb") as f:
    pickle.dump(noncompliance_scores, f)
print("DONE: noncompliance scores")

# Print first 10 noncompliance scores
print("First 10 noncompliance scores:")
for k in list(noncompliance_scores)[:10]:
    print(k, "=>", noncompliance_scores[k])

# Save autopsy scores
autopsy_scores = dict(zip(chartevents_ids, autopsy_svm.decision_function(chartevents_X)))
with open(os.path.join(base_path, "mistrust_autopsy.pkl"), "wb") as f:
    pickle.dump(autopsy_scores, f)
print("DONE: autopsy scores")

# Print first 10 autopsy scores
print("First 10 autopsy scores:")
for k in list(autopsy_scores)[:10]:
    print(k, "=>", autopsy_scores[k])


DONE: noncompliance scores
First 10 noncompliance scores:
131072 => -4.741353745870147
131073 => -4.504105222026098
131076 => -4.848367575153781
131077 => -5.735136830922424
131078 => -3.2965189720272883
131082 => -3.705597803351222
131084 => -5.427220213054145
131085 => -4.4404302118768975
131086 => -4.86731513875173
131087 => -4.462367867747527
DONE: autopsy scores
First 10 autopsy scores:
131072 => -1.021530079978897
131073 => -0.6105532905547417
131076 => -1.021530079978897
131077 => -0.6105532905547417
131078 => -1.1919214810478986
131082 => -1.029058826618701
131084 => -1.3165573430225783
131085 => -0.7256938807732484
131086 => -1.021530079978897
131087 => -1.028265419676179


<h1>Sentiment Analysis as Mistrust Proxy</h1>

In [ ]:
#original sentiment analysis code redacted - output is left for comparison

59568it [26:58, 36.80it/s]


sa: 52726
DONE: negative sentiment scores


In [ ]:
!pip install git+https://github.com/clips/pattern

  Cloning https://github.com/clips/pattern to /tmp/pip-req-build-c47240oc
  Running command git clone --filter=blob:none --quiet https://github.com/clips/pattern /tmp/pip-req-build-c47240oc
  Resolved https://github.com/clips/pattern to commit d25511f9ca7ed9356b801d8663b8b5168464e68f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.8/349.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 7.4 MB/s eta 0:00:00
  Created wheel for Pa

In [ ]:
from pattern.en import parse

In [ ]:
from pattern.en import sentiment
import os
import tqdm
import numpy as np
import pickle
from google.colab import drive

# Mount Google Drive
# drive.mount('/content/drive')
base_path = "/content/drive/MyDrive/mimic_data/data"
os.makedirs(base_path, exist_ok=True)

# Query discharge summaries only
disch_query = f"""
SELECT DISTINCT n.hadm_id, n.category, n.text, n.chartdate, n.charttime
FROM physionet-data.{note_dataset_3}.noteevents n
WHERE iserror IS NULL
  AND hadm_id IS NOT NULL
  AND category = 'Discharge summary'
"""
disch = run_query(disch_query)

# Compute sentiment polarity scores (normalized)
sentiments = {}
for _, row in tqdm.tqdm(disch.iterrows(), total=disch.shape[0]):
    hadm_id = row.hadm_id
    text = row.text

    # Pattern's sentiment returns (polarity, subjectivity)
    polarity, _ = sentiment(text.split())  # Faster and avoids pattern's full parse tree
    sentiments[hadm_id] = polarity

# Normalize sentiment scores (z-score)
scores = np.array(list(sentiments.values()))
mu, std = scores.mean(), scores.std()
sentiments = {k: -(v - mu) / std for k, v in sentiments.items()}

print("sa:", len(sentiments))

# Save normalized negative sentiment scores
with open(os.path.join(base_path, "neg_sentiment.pkl"), "wb") as f:
    pickle.dump(sentiments, f)

print("DONE: negative sentiment scores")


100%|██████████| 59568/59568 [10:23<00:00, 95.52it/s]


sa: 52726
DONE: negative sentiment scores


In [ ]:
import os
import pickle

with open(os.path.join(base_path, "neg_sentiment.pkl"), "rb") as f:
    sentiments = pickle.load(f)

positive_count = sum(1 for val in sentiments.values() if val > 0)
negative_count = sum(1 for val in sentiments.values() if val < 0)

print("Positive:", positive_count)
print("Negative:", negative_count)

Positive: 26887
Negative: 25839


In [43]:
#LLM for sentiment analysis

import os
import tqdm
import numpy as np
import pickle
from openai import OpenAI
import pandas as pd
from google.colab import drive

# Mount Google Drive
base_path = "/content/drive/MyDrive/mimic_data/data"
os.makedirs(base_path, exist_ok=True)

client = OpenAI(
    api_key='your-open-ai-key')

# Query discharge summaries only
disch_query = """
SELECT DISTINCT n.hadm_id, n.category, n.text, n.chartdate, n.charttime
FROM physionet-data.mimiciii_notes.noteevents n
WHERE iserror IS NULL
  AND hadm_id IS NOT NULL
  AND category = 'Discharge summary'
"""
disch = run_query(disch_query)

# Limit or clean for GPT use
disch = disch.dropna(subset=["text"])

In [62]:
import re

sentiments = {}

gpt_sentiment_path = f"{base_path}/gpt_sentiment"
os.makedirs(gpt_sentiment_path, exist_ok=True)

BATCH_SIZE = 100

def format_batch_prompt(hadm_ids, texts):
    prompt = [
        "You are a medical assistant. Evaluate the snippet of a hospital discharge note for the level of patient mistrust in the medical procedure and clinician.\n",
        "Evaluate on a scale of -1.00 to 1.00. A score of 1.00 means that the patient is in active defiance of medical advice. A score of -1.00 means that a patient is fully compliant with all medical advice. ",
        "Return only a numbered list of scores, one per summary. Format:\n1. 0.34\n2. -0.80\n..."
    ]
    for i, (hid, txt) in enumerate(zip(hadm_ids, texts), 1):
        snippet = txt.replace("\n", " ").strip()
        prompt.append(f"\n{i}. HADM_ID {hid}: {snippet}")
    return "\n".join(prompt)

def parse_scores(response_text, hadm_ids):
    results = {}
    for line in response_text.strip().splitlines():
        try:
            idx_str, val_str = line.strip().split(".", 1)
            idx = int(idx_str.strip()) - 1
            val = float(val_str.strip())
            if 0 <= idx < len(hadm_ids):
                results[hadm_ids[idx]] = val
        except Exception:
            continue
    return results

filtered_rows = []
brief_texts = []
for idx, row in disch.iterrows():
    match_text = re.search(r"Brief Hospital Course:(.*?)(?:\n[A-Z][^\n]*?:)", row.text, re.DOTALL)
    if match_text and match_text.group(1).strip() != '':
        filtered_rows.append(row)
        brief_texts.append(match_text.group(1).strip())

disch = pd.DataFrame(filtered_rows)
disch["brief_hospital_course"] = brief_texts


In [65]:
for start in tqdm.trange(0, len(disch), BATCH_SIZE):
    batch = disch.iloc[start:start + BATCH_SIZE]
    hadm_ids = batch["hadm_id"].tolist()
    texts = batch["brief_hospital_course"].tolist()
    prompt = format_batch_prompt(hadm_ids, texts)

    try:
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=prompt,
            temperature=0.0,
            max_output_tokens=2000,
        )
        output_text = response.output_text.strip()
        batch_scores = parse_scores(output_text, hadm_ids)
        with open(os.path.join(gpt_sentiment_path, f"neg_sentiment_gpt_batched_{start}.pkl"), "wb") as f:
          pickle.dump(batch_scores, f)
        sentiments.update(batch_scores)
    except Exception as e:
        print(f"Error in batch starting at {start}: {e}")

# Normalize sentiment scores (z-score)
'''
scores = np.array(list(sentiments.values()))
mu, std = scores.mean(), scores.std()
sentiments = {k: -(v - mu) / std for k, v in sentiments.items()}
'''

# Save to pickle
with open(os.path.join(base_path, "neg_sentiment_gpt_batched.pkl"), "wb") as f:
    pickle.dump(sentiments, f)

print("✅ DONE: Batched GPT-based sentiment scoring")

100%|██████████| 347/347 [1:42:35<00:00, 17.74s/it]

✅ DONE: Batched GPT-based sentiment scoring


In [72]:
import os
import pickle

with open(os.path.join(base_path, "neg_sentiment_gpt_batched.pkl"), "rb") as f:
    hot_loaded_sentiments = pickle.load(f)

positive_count = sum(1 for val in hot_loaded_sentiments.values() if val > 0)
negative_count = sum(1 for val in hot_loaded_sentiments.values() if val < 0)

print("Positive:", positive_count)
print("Negative:", negative_count)
print(positive_count/(positive_count+negative_count))
print(hot_loaded_sentiments)

Positive: 2960
Negative: 31029
0.08708699873488482
{114823: -0.8, 175058: -0.9, 113103: -0.95, 147438: -0.85, 189690: -0.75, 152105: -0.7, 140451: -0.7, 183978: -0.8, 122706: -0.75, 142690: -0.8, 113611: -0.85, 102957: -0.8, 138697: -0.75, 114333: -0.8, 175640: -0.7, 134184: -0.75, 132534: -0.85, 141268: -0.9, 199580: -0.7, 123581: -0.85, 118719: -0.75, 109321: -0.8, 184132: -0.7, 175597: -0.75, 109800: -0.8, 186236: -0.75, 158090: -0.8, 192458: -0.75, 135501: -0.7, 120148: -0.9, 168109: -0.8, 172850: -0.85, 112194: -0.9, 104140: -0.8, 110224: -0.75, 145677: -0.75, 139627: -0.8, 155312: -0.85, 195564: -0.9, 124457: -0.85, 130751: -0.8, 108944: -1.0, 108489: -0.85, 139284: -0.75, 135801: -0.8, 183566: -0.75, 104646: -0.85, 171872: -0.8, 152258: -0.75, 135295: -0.8, 125747: -0.75, 119471: -0.8, 133664: -0.05, 186066: -0.75, 118841: 0.9, 179698: -0.75, 191698: -0.8, 175578: -0.7, 102092: -0.85, 196410: -0.75, 166159: -0.8, 148571: -0.85, 132305: -0.9, 162363: -0.8, 161668: -0.75, 178662: 